In [1]:
using Combinatorics
using IterativeSolvers  # for LSQR
const IntMat = SparseMatrixCSC{Float64,Int64}
const FloatMat = SparseMatrixCSC{Float64,Int64}

SparseMatrixCSC{Float64,Int64}

In [2]:
# function to read .paj files (THANKS BRAD NELSON for writing this!)
function read_paj_sparse(fname::AbstractString)
    n = 0
    local A, I, J, V
    local labels
    open(fname) do f
        # get number of vertices                                                                                                                                                     
        while true
            line = readline(f)
            if length(line) > 8 && line[1:9] == "*vertices"
                n = parse(Int64, line[10:end])
                break
            end
        end

        # get labels                                                                                                                                                                 
        labels = Array{String}(n)
        while true
            line = readline(f)
            if length(line) >=9 && line[1:9] == "*vertices"; break; end
        end
        for i = 1:n
            line = readline(f)
            labels[i] = line[7:end-1]
        end

        # get edges                                                                                                                                                                  
        I, J, V = Array{Int64}(0), Array{Int64}(0), Array{Float64}(0)
        while true
            line = readline(f)
            if length(line) > 4 && line[1:5] == "*arcs"; break; end
        end
        while true
            line = readline(f)
            if length(line) < 2; break; end
            (is, js, vs) = split(line)
            push!(I, parse(Int64, is))
            push!(J, parse(Int64, js))
            push!(V, parse(Float64, vs))
        end
    end
    A = convert(FloatMat, sparse(I, J, V, n, n))
    return (A, labels)
end

read_paj_sparse (generic function with 1 method)

In [3]:
# gradient matrix from graph adjacency matrix
function grad_mat(A::FloatMat)
    edge_map = Dict{NTuple{2, Int64}, Int64}()
    I, J, V = Int64[], Int64[], Int64[]
    curr_edge_ind = 1
    num_vertices = size(A, 2)
    for j in 1:num_vertices, i in find(A[:,j])
        if i < j
            edge_map[(i, j)] = curr_edge_ind
            push!(I, curr_edge_ind, curr_edge_ind)
            push!(J, i, j)
            push!(V, -1, 1)
            curr_edge_ind += 1
        end
    end
    grad = convert(FloatMat, sparse(I, J, V, length(edge_map), num_vertices))
    return (grad, edge_map)
end

grad_mat (generic function with 1 method)

In [4]:
# curl matrix from graph adjacency matrix
# edge_map takes (i, j) --> index 1, 2, ..., #edges
function curl_mat(A::FloatMat, edge_map::Dict{NTuple{2, Int64}, Int64})
    num_vertices = size(A, 2)
    # ordering of nodes by degree
    deg_order = zeros(Int64, num_vertices)
    deg_order[sortperm(vec(sum(spones(A), 1)))] = collect(1:num_vertices)
    tri_map = Dict{NTuple{3, Int64}, Int64}()
    curr_tri_ind = 1
    I, J, V = Int64[], Int64[], Int64[]
    for i in 1:num_vertices
        pos = deg_order[i]
        neighbors = filter(v -> deg_order[v] > pos, find(A[:, i]))
        for (j, k) in combinations(neighbors, 2)
            if A[j, k] > 0.0
                # triangle {i, j, k}
                a, b, c = sort([i, j, k])
                tri_map[(a, b, c)] = curr_tri_ind
                push!(I, curr_tri_ind, curr_tri_ind, curr_tri_ind)
                push!(J, edge_map[(a, b)], edge_map[(a, c)], edge_map[(b, c)])
                push!(V, 1, -1, 1)
                curr_tri_ind += 1
            end
        end
    end
    curl = convert(FloatMat, sparse(I, J, V, length(tri_map), length(edge_map)))
    return (curl, tri_map)
end

curl_mat (generic function with 1 method)

In [5]:
# hodge decomposition for an edge flow on a graph
function hodge_decomp(X::Vector{Float64}, grad::FloatMat, curl::FloatMat)
    assert(size(grad, 1) == length(X) == size(curl, 2))
    f = lsqr(grad, X, atol=1e-10, btol=1e-10, conlim=1e12)
    Φ = lsqr(curl', X, atol=1e-10, btol=1e-10, conlim=1e12)
    X_H = X - curl' * Φ - grad * f
    return (f, Φ, X_H)
end

hodge_decomp (generic function with 1 method)

In [6]:
# read data
@time A, labels = read_paj_sparse("Florida.paj")

# form gradient and curl matrices
Asym = max.(A, A') # symmetrize A to be undirected graph
@time (grad, edge_map) = grad_mat(Asym)
@time (curl, tri_map) = curl_mat(Asym, edge_map)

# santiy check: make sure A * B = 0
curlgrad = curl * grad
println("min(curl grad) = $(minimum(curlgrad)), max(curl grad) = $(maximum(curlgrad))")

  0.908948 seconds (353.63 k allocations: 19.958 MiB, 2.33% gc time)
  0.563634 seconds (104.66 k allocations: 6.438 MiB)
  1.740330 seconds (445.06 k allocations: 29.273 MiB, 1.75% gc time)
min(curl grad) = 0.0, max(curl grad) = 0.0


In [7]:
# form an edge from uppper triangular part of matrix and ordering of edges
function edge_flow(T::FloatMat, edge_map::Dict{NTuple{2, Int64}, Int64})
    X = zeros(length(edge_map))
    B = triu(T, 1)
    for j in 1:size(B, 2), i in find(B[:,j])
        X[edge_map[(i, j)]] = B[i, j]
    end
    return X
end
@time X = edge_flow(A - A', edge_map) # takes asymmetry into account

# decompose edge flow 
@time (f, Φ, X_H) = hodge_decomp(X, grad, curl)

# sanity check: are our components orthogonal?
x1 = grad * f
x2 = curl' * Φ
@show x1' * x2, x1' * X_H, x2' * X_H

  0.218300 seconds (41.69 k allocations: 2.421 MiB)
  2.345464 seconds (828.50 k allocations: 45.366 MiB, 1.35% gc time)
(x1' * x2, x1' * X_H, x2' * X_H) = (-1.8189894035458565e-12, 2.020428269133845e-11, 2.2275435185292736e-7)


(-1.8189894035458565e-12, 2.020428269133845e-11, 2.2275435185292736e-7)

In [8]:
# get potential function ranking
ranking = labels[sortperm(f), rev=true]
@show ranking[1:5]
@show ranking[end-4:end]

# how much of norm due to gradient potential?
frac_potential = norm(grad * f, 2)^2 / norm(X, 2)^2
@show frac_potential

ranking[1:5] = String["Input", "Roots", "DOC", "Drift Algae", "Oscillatoria"]
ranking[end - 4:end] = String["Lizardfish", "Brotalus", "Predatory Crabs", "Mackerel", "Respiration"]
frac_potential = 0.10523817822464862


0.10523817822464862

In [9]:
# only consider sign of edge
X = edge_flow(sign.(A - A'), edge_map)
@time (f, Φ, X_H) = hodge_decomp(X, grad, curl)
# get potential function ranking
ranking = labels[sortperm(f), rev=true]

# get potential function ranking
ranking = labels[sortperm(f)]
@show ranking[1:5]
@show ranking[end-4:end]
frac_potential = norm(grad * f, 2)^2 / norm(X, 2)^2
@show frac_potential

  0.033038 seconds (890 allocations: 989.031 KiB)
ranking[1:5] = String["Input", "Small Diatoms (<20um)", "Benthic Phytoplankton", "2um Spherical Phytoplankt", "Synedococcus"]
ranking[end - 4:end] = String["Dolphin", "Respiration", "Crocodiles", "Output", "Raptors"]
frac_potential = 0.7872575994341737


0.7872575994341737

In [10]:
# only consider asymmetric flows
A1 = spones(A)
B = A1 .* A1' # bi-directional flows
U = A1 - B

X = edge_flow(sign.(U - U'), edge_map)
@time (f, Φ, X_H) = hodge_decomp(X, grad, curl)

# get potential function ranking
ranking = labels[sortperm(f)]
@show ranking[1:5]
@show ranking[end-4:end]
frac_potential = norm(grad * f, 2)^2 / norm(X, 2)^2
@show frac_potential

  0.038877 seconds (890 allocations: 989.031 KiB)
ranking[1:5] = String["Input", "Small Diatoms (<20um)", "Benthic Phytoplankton", "Synedococcus", "2um Spherical Phytoplankt"]
ranking[end - 4:end] = String["Dolphin", "Respiration", "Crocodiles", "Output", "Raptors"]
frac_potential = 0.7958274287806822


0.7958274287806822

In [11]:
# find inconsistent triangles
ordered_tris = sort(collect(keys(tri_map)), by=(k -> Φ[tri_map[k]]), rev=true)
for i = 1:5
    @show labels[[ordered_tris[i]...]]
end

labels[[ordered_tris[i]...]] = String["Dinoflagellates", "Meroplankton", "Input"]
labels[[ordered_tris[i]...]] = String["Meroplankton", "Water POC", "Input"]
labels[[ordered_tris[i]...]] = String["Epiphytes", "Thor Floridanus", "Water POC"]
labels[[ordered_tris[i]...]] = String["Oscillatoria", "Echinoderma", "Parrotfish"]
labels[[ordered_tris[i]...]] = String["Coral", "Parrotfish", "Respiration"]


In [12]:
pf = find(labels .== "Parrotfish")[1]
@show labels[find(A[:,pf])]
@show labels[find(A[pf,:])]

labels[find(A[:, pf])] = String["Synedococcus", "Oscillatoria", "Drift Algae", "Epiphytes", "Meiofauna", "Sponges", "Coral", "Echinoderma", "Bivalves", "Detritivorous Polychaetes", "Predatory Polychaetes", "Suspension Feeding Polych", "Macrobenthos"]
labels[find(A[pf, :])] = String["Tarpon", "Barracuda", "Other Pelagic Fishes", "Loon", "Greeb", "Pelican", "Comorant", "Predatory Ducks", "Raptors", "Crocodiles", "Dolphin", "Water POC", "Output", "Respiration"]


14-element Array{String,1}:
 "Tarpon"              
 "Barracuda"           
 "Other Pelagic Fishes"
 "Loon"                
 "Greeb"               
 "Pelican"             
 "Comorant"            
 "Predatory Ducks"     
 "Raptors"             
 "Crocodiles"          
 "Dolphin"             
 "Water POC"           
 "Output"              
 "Respiration"         

In [14]:
A, labels = read_paj_sparse("Michigan.paj")
Asym = max.(A, A') # symmetrize A to be undirected graph
@time (grad, edge_map) = grad_mat(Asym)
@time (curl, tri_map) = curl_mat(Asym, edge_map)
# only consider asymmetric flows
A1 = spones(A)
B = A1 .* A1' # bi-directional flows
U = A1 - B

X = edge_flow(sign.(U - U'), edge_map)
@time (f, Φ, X_H) = hodge_decomp(X, grad, curl)

# get potential function ranking
ranking = labels[sortperm(f)]
@show ranking[1:5]
@show ranking[end-4:end]
frac_potential = norm(grad * f, 2)^2 / norm(X, 2)^2
@show frac_potential

  0.000125 seconds (215 allocations: 101.047 KiB)
  0.000594 seconds (4.33 k allocations: 433.453 KiB)
  0.001084 seconds (484 allocations: 68.516 KiB)
ranking[1:5] = String["Input", "Flagellates", "Blue-greenGree", "Diatoms", "Calanoids"]
ranking[end - 4:end] = String["Juv. Lake Trout", "Detritus", "Sea lamprey", "Respiration", "Output"]
frac_potential = 0.7025775654156604


0.7025775654156604